In [123]:
import pandas as pd
import cvxpy as cp
import numpy as  np
from mamimo.time_utils import add_time_features, PowerTrend
from mamimo.carryover import ExponentialCarryover
from mamimo.saturation import ExponentialSaturation
import pandas as pd
import numpy as np

from mamimo.time_utils import add_time_features, PowerTrend
from mamimo.carryover import ExponentialCarryover
from mamimo.saturation import ExponentialSaturation

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin

from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import holidays

import seaborn as sns
import matplotlib.pyplot as plt

import math
from sklearn.preprocessing import MinMaxScaler

In [3]:
df=pd.read_csv('../../raw_data/df.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)


In [4]:
df['Day'] = pd.to_datetime(df['Day'])
df.set_index('Day', inplace=True)

In [5]:
df = df.sort_values(by='Day')

In [6]:
df = df.rename(columns={"fb_costs": "facebook", "google_costs": "google", "tt_costs": "tiktok"})
df['day'] = df.index # we will use this in our column transformer
df = df.drop(columns = ['fb_impressions', 'fb_clicks', 'google_impressions', 'google_clicks', 'tt_impressions', 'tt_clicks'])
df

,orders,total_sales,facebook,google,tiktok,day
Day,,,,,,
2021-07-01,10,414.85,257.01,1.17,0.00,2021-07-01
2021-07-02,6,229.95,250.30,0.84,0.00,2021-07-02
2021-07-03,10,459.89,248.59,0.72,0.00,2021-07-03
2021-07-04,17,508.36,258.05,0.07,0.00,2021-07-04
2021-07-05,13,399.88,257.47,0.38,0.00,2021-07-05
...,...,...,...,...,...,...
2023-08-20,294,13616.97,2148.53,466.56,368.86,2023-08-20
2023-08-21,356,15348.42,2210.01,400.58,391.14,2023-08-21
2023-08-22,150,6054.53,1335.84,289.46,218.61,2023-08-22


In [1]:
def minmax_fb(x):
    scaled_fb = ((x - 0)/(2294.69 - 0))
    return scaled_fb
def minmax_gg(x):
    scaled_gg = ((x - 0)/(480.66 - 0))
    return scaled_gg
def minmax_tt(x):
    scaled_tt = ((x - 0)/(840.37- 0))
    return scaled_tt

In [2]:
minmax_fb(5)

0.0021789435610038827

In [35]:
n_splits = 5  # Number of splits for cross-validation
tscv = TimeSeriesSplit(n_splits=n_splits)

for train_index, test_index in tscv.split(df):
    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]

y_train = train_df['total_sales']
X_train = train_df.drop(columns = ['total_sales', 'orders'])


y_test = test_df['total_sales']
X_test = test_df.drop(columns = ['total_sales', 'orders'])

In [36]:
X_train

,facebook,google,tiktok,day
Day,,,,
2021-07-01,257.01,1.17,0.00,2021-07-01
2021-07-02,250.30,0.84,0.00,2021-07-02
2021-07-03,248.59,0.72,0.00,2021-07-03
2021-07-04,258.05,0.07,0.00,2021-07-04
2021-07-05,257.47,0.38,0.00,2021-07-05
...,...,...,...,...
2023-04-12,885.33,208.52,209.03,2023-04-12
2023-04-13,882.17,186.47,208.82,2023-04-13
2023-04-14,662.08,160.69,216.38,2023-04-14


In [9]:
X_train["facebook"] = X_train["facebook"].map(lambda x: math.log(x) if x != 0 else 0)

In [10]:
X_train["google"] = X_train["google"].map(lambda x: math.log(x) if x != 0 else 0)

In [11]:
X_train["tiktok"] = X_train["tiktok"].map(lambda x: math.log(x) if x != 0 else 0)

In [37]:
X_train.drop(columns=['day'], inplace=True)

In [14]:
X_test["facebook"] = X_test["facebook"].map(lambda x: math.log(x) if x != 0 else 0)

In [15]:
X_test["google"] = X_test["google"].map(lambda x: math.log(x) if x != 0 else 0)

In [16]:
X_test["tiktok"] = X_test["tiktok"].map(lambda x: math.log(x) if x != 0 else 0)

In [40]:
X_test.drop(columns=['day'], inplace=True)

In [25]:
y_test

Day
2023-04-17     3121.57
2023-04-18     3851.15
2023-04-19     2858.61
2023-04-20     3623.44
2023-04-21     1804.11
                ...   
2023-08-20    13616.97
2023-08-21    15348.42
2023-08-22     6054.53
2023-08-23     4810.67
2023-08-24     4889.20
Name: total_sales, Length: 130, dtype: float64

In [38]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [41]:
model.score(X_test, y_test)

0.31491390570371014

In [73]:
! pip install cvxpy

  Obtaining dependency information for cvxpy from https://files.pythonhosted.org/packages/ed/42/25b2cacd49a10547d36c530df400da1729511dbc8f5d725712130038cc65/cvxpy-1.3.2-cp310-cp310-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for osqp>=0.4.1 from https://files.pythonhosted.org/packages/4c/5f/7fc8e85b247b473f7e787fefa8eb7b1b6357f4e3c4ef2d5932f0fac49028/osqp-0.6.3-cp310-cp310-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 8.5 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for setuptools>65.5.1 from https://files.pythonhosted.org/packages/4f/ab/0bcfebdfc3bfa8554b2b2c97a555569c4c1ebc74ea288741ea8326c51906/setuptools-68.1.2-py3-none-any.whl.metadata
  Using cached setuptools-68.1.2-py3-none-any.whl.metadata (6.2 kB)
  Obtaining dependency information for qdldl from https://files.pythonhosted.org/packages/7b/e9/d995f55cb7599

In [99]:
df=pd.DataFrame([pd.to_datetime('2023-09-05')])
df.set_index(0, inplace=True)
df['day']='2023-09-05'
df['day']=pd.to_datetime(df['day'])

In [100]:
class AddHolidaysTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn any parameters during fitting,
        # so we simply return self.
        return self

    def transform(self, X):
        """Creates a new column with row value = 1 if the day is a Friday or Saturday and 0 if not."""
        df = X.copy()  # Create a copy of the input DataFrame to avoid modifying it directly
        de_holiday_list = []
        for holiday in holidays.Germany(years=[2021,2022,2023]).items():
            de_holiday_list.append(holiday)
        de_holidays_df = pd.DataFrame(de_holiday_list, columns=["date", "holiday"])
        de_holidays_df['date'] = pd.to_datetime(de_holidays_df['date'])
        de_holidays_df.set_index('date', inplace=True)
        
        at_holiday_list = []
        for holiday in holidays.Austria(years=[2021,2022,2023]).items():
            at_holiday_list.append(holiday)
        at_holidays_df = pd.DataFrame(at_holiday_list, columns=["date", "holiday"])
        at_holidays_df['date'] = pd.to_datetime(at_holidays_df['date'])
        at_holidays_df.set_index('date', inplace=True)

        # add DE holidays to df
        merged_df = df.merge(de_holidays_df, how='left', left_index=True, right_index=True)
        merged_df['de_holiday'] = merged_df.index.isin(de_holidays_df.index).astype(int)
        merged_df.drop(columns=['holiday'], inplace=True)
        
        # add AT holidays to df
        at_holidays_df['at_holiday'] = 1 # add a 1 column to austrian holidays dataframe to help us merge with DE holidays
        merged_df = merged_df.merge(at_holidays_df[['at_holiday']], how='left', left_index=True, right_index=True)
        merged_df.head() # creates two columns (at_holiday_x, at_holiday_y), we only need one
        merged_df['at_holiday'].fillna(0, inplace=True) # replace NaN (no holiday) with 0
        merged_df['at_holiday'] = merged_df['at_holiday'].astype(int) # convert 1 and 0 to integers
        
        # combine columns
        merged_df['holiday'] = (merged_df['at_holiday'] | merged_df['de_holiday']).astype(int)
        merged_df = merged_df.drop(columns = ['de_holiday', 'at_holiday']) # drop individual DE and AT rows
        
        return merged_df[['holiday']]#.reset_index(drop=True)


In [101]:
class AddWeekendsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn any parameters during fitting,
        # so we simply return self.
        return self

    def transform(self, X):
        """Creates a new column with row value = 1 if the day is a Friday or Saturday and 0 if not."""
        df = X.copy()  # Create a copy of the input DataFrame to avoid modifying it directly
        weekday_values = df.index.weekday
        df['fri_sat'] = ((weekday_values == 4) | (weekday_values == 5)).astype(int)
        return df[['fri_sat']]

In [102]:
class AddMonthsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn any parameters during fitting,
        # so we simply return self.
        return self

    def transform(self, X):
        df = X.copy()  # Create a copy of the input DataFrame to avoid modifying it directly
        df = (df
        .pipe(add_time_features, month=True)
        )
        
        months_in_a_year = 12

        df['sin_MonthYear'] = np.sin(2*np.pi*(df['month'])/months_in_a_year)
        df['cos_MonthYear'] = np.cos(2*np.pi*(df['month'])/months_in_a_year)
        df.drop(columns=['month'], inplace=True)
        
        return df[['sin_MonthYear', 'cos_MonthYear']]

In [103]:
df

,day
0,
2023-09-05,2023-09-05


In [104]:
time_features = ColumnTransformer(
    [
     ('holidays_pipe', Pipeline([
            ('add_holidays', AddHolidaysTransformer())
     ]), ['day']),
     ('weekends_pipe', Pipeline([
            ('add_weekends', AddWeekendsTransformer())
     ]), ['day']),
     ('months_pipe', Pipeline([
            ('add_months', AddMonthsTransformer())
     ]), ['day'])])

In [105]:
transformed_df = time_features.fit_transform(df)
transformed_df = pd.DataFrame(transformed_df, columns = ['holiday', 'weekends', 'sin', 'cos'])
transformed_df

,holiday,weekends,sin,cos
0,0.0,0.0,-1.0,-1.836970e-16


In [120]:
TOTAL_DAILY_BUDGET= 5000


In [131]:

import cvxpy as cp

# Variables
google   = cp.Variable(pos=True)
facebook = cp.Variable(pos=True)
tiktok  = cp.Variable(pos=True)

# Constraint
constraint = [google + facebook + tiktok <= TOTAL_DAILY_BUDGET, google>=TOTAL_DAILY_BUDGET*0.04769543069031596, facebook>=TOTAL_DAILY_BUDGET*0.16612285992471154, tiktok>=TOTAL_DAILY_BUDGET*0.0361817093849725]
              
betas=[-164.052961,-372.686127,265.076699,-23.341808,1.144179,5.504117,1.483814]
intercept=238.56530283612096
# Objective

min_ma=MinMaxScaler()
obj= cp.Maximize(intercept+betas[0] * transformed_df['holiday'][0] +betas[1] * transformed_df['weekends'][0] +betas[2] *transformed_df['sin'][0]+ betas[3] *transformed_df['cos'][0]+ betas[4] *(facebook)+betas[5] *(google)+betas[5] *(tiktok))
                 

In [132]:
%%time

# Solve
prob = cp.Problem(obj, constraint)
prob.solve(solver='ECOS', verbose=False)

# Print solution
print('='*59 + '\n' + ' '*24 + 'Solution' + ' '*24 + '\n' + '='*59)
print(f'Status = {prob.status}')
print(f'Returns = ${round(prob.value):,}\n')
print('Marketing allocation:')
print(f' - Google Ads   = ${round(google.value):,}')
print(f' - Facebook Ads = ${round(facebook.value):,}')
print(f' - Tiktok Ads  = ${round(tiktok.value):,}')

                        Solution                        
Status = optimal
Returns = $23,873

Marketing allocation:
 - Google Ads   = $2,097
 - Facebook Ads = $831
 - Tiktok Ads  = $2,072
CPU times: user 17.4 ms, sys: 44.6 ms, total: 61.9 ms
Wall time: 152 ms
